In [ ]:
import pandas as pd
import numpy as np
import re

# === CONFIGURATION ===
INPUT_FILE  = "5sznset_final.xlsx"           # your raw merged panel file
OUTPUT_FILE = "5sznset_cleaned_full.xlsx"    # cleaned, enriched output

# === HELPER FUNCTIONS ===
def clean_text(s):
    """Lowercase, strip, collapse whitespace."""
    return re.sub(r'\s+', ' ', str(s).lower().strip())

def map_series(series, mapping):
    """Apply clean_text + mapping dict, then restore title‐case if needed."""
    return (series.fillna('')
                  .apply(clean_text)
                  .map(mapping)
                  .fillna(series)
                  .astype(str))

# === FULL MAPPING DICTIONARIES ===
# (Copied from your code above; add/remove entries as you refine)
nation_mapping = {
    "eng eng": "England", "es esp": "Spain", "ie irl": "Ireland", "fr fra": "France",
    "ma mar": "Morocco", "dz alg": "Algeria", "eg egy": "Egypt", "tn tun": "Tunisia",
    "sa ksa": "Saudi Arabia", "de ger": "Germany", "it ita": "Italy", "pt por": "Portugal",
    "nl ned": "Netherlands", "be bel": "Belgium", "ch sui": "Switzerland", "at aut": "Austria",
    "dk den": "Denmark", "no nor": "Norway", "se swe": "Sweden", "pl pol": "Poland",
    "cz cze": "Czech Republic", "sk svk": "Slovakia", "hu hun": "Hungary",
    "ro rou": "Romania", "hr cro": "Croatia", "rs srb": "Serbia",
    "ba bih": "Bosnia and Herzegovina", "bosnia-herzegovina": "Bosnia and Herzegovina",
    "tr tur": "Turkey", "gr gre": "Greece", "us usa": "United States", "ca can": "Canada",
    "mx mex": "Mexico", "ar arg": "Argentina", "br bra": "Brazil", "co col": "Colombia",
    "uy uru": "Uruguay", "cl chi": "Chile", "pe per": "Peru", "gh gha": "Ghana",
    "ci civ": "Ivory Coast", "cote d'ivoire": "Ivory Coast", "cm cmr": "Cameroon",
    "sn sen": "Senegal", "ng nga": "Nigeria", "za rsa": "South Africa",
    "jp jpn": "Japan", "cn chn": "China", "kr kor": "South Korea", "ir irn": "Iran",
    "au aus": "Australia", "ph phi": "Philippines", "pr pur": "Puerto Rico",
    "lt ltu": "Lithuania", "gp glp": "Guadeloupe", "mt mlt": "Malta", "ml mli": "Mali",
    "gn gui": "Guinea", "gm gam": "The Gambia", "uz uzb": "Uzbekistan", "me mne": "Montenegro",
    "is isl": "Iceland", "ru rus": "Russia", "al alb": "Albania", "ua ukr": "Ukraine",
    "iq irq": "Iraq", "ly lby": "Libya", "sct sco": "Scotland", "cg cgo": "Congo",
    "cd cod": "Democratic Republic Of The Congo", "py par": "Paraguay", "xk kvx": "Kosovo",
    "nz nzl": "New Zealand", "si svn": "Slovenia", "gw gnb": "Guinea-Bissau",
    "mk mkd": "North Macedonia", "wls wal": "Wales", "ge geo": "Georgia",
    "ht hai": "Haiti", "ao ang": "Angola", "nir nir": "Northern Ireland", "bj ben": "Benin",
    "cv cpv": "Cape Verde", "ve ven": "Venezuela", "tg tog": "Togo", "gf guf": "French Guiana",
    "bf bfa": "Burkina Faso", "jm jam": "Jamaica", "fi fin": "Finland", "cf cta": "Central African Republic",
    "cy cyp": "Cyprus", "am arm": "Armenia", "ga gab": "Gabon", "id idn": "Indonesia",
    "ec ecu": "Ecuador", "gq eqg": "Equatorial Guinea", "sl sle": "Sierra Leone",
    "ee est": "Estonia", "zm zam": "Zambia", "il isr": "Israel"
}

club_mapping = {
    'angers':'Angers Sco','arsenal':'Arsenal Fc','atalanta':'Atalanta Bc',
    'athletic club':'Athletic Bilbao','atletico madrid':'Atlético De Madrid',
    'augsburg':'Fc Augsburg','auxerre':'Aj Auxerre','barcelona':'Fc Barcelona',
    'bayer leverkusen':'Bayer 04 Leverkusen','bochum':'Vfl Bochum',
    'bologna':'Bologna Fc 1909','bournemouth':'Afc Bournemouth','brentford':'Brentford Fc',
    'brest':'Stade Brestois 29','brighton':'Brighton & Hove Albion','cagliari':'Cagliari Calcio',
    'celta vigo':'Celta De Vigo','chelsea':'Chelsea Fc','como':'Como 1907','empoli':'Fc Empoli',
    'espanyol':'Rcd Espanyol Barcelona','everton':'Everton Fc','fiorentina':'Acf Fiorentina',
    'freiburg':'Sc Freiburg','fulham':'Fulham Fc','genoa':'Genoa Cfc','getafe':'Getafe Cf',
    'girona':'Girona Fc','heidenheim':'1.Fc Heidenheim 1846','hoffenheim':'Tsg 1899 Hoffenheim',
    'juventus':'Juventus Fc','las palmas':'Ud Las Palmas','lazio':'Ss Lazio',
    'le havre':'Le Havre Ac','lecce':'Us Lecce','leganes':'Cd Leganés','leicester':'Leicester City',
    'leipzig':'Rb Leipzig','lens':'Rc Lens','lille':'Losc Lille','liverpool':'Liverpool Fc',
    'lyon':'Olympique Lyon','mainz':'1.Fsv Mainz 05','mallorca':'Rcd Mallorca',
    'marseille':'Olympique Marseille','monaco':'As Monaco','monchengladbach':'Borussia Mönchengladbach',
    'montpellier':'Montpellier Hsc','monza':'Ac Monza','nantes':'Fc Nantes','napoli':'Ssc Napoli',
    'newcastle':'Newcastle United','nice':'Ogc Nice','osasuna':'Ca Osasuna','parma':'Parma Calcio 1913',
    'psg':'Paris Saint-Germain','real betis':'Real Betis Balompié','reims':'Stade Reims','rennes':'Stade Rennais Fc',
    'roma':'As Roma','sevilla':'Sevilla Fc','southampton':'Southampton Fc','st pauli':'Fc St. Pauli',
    'st-etienne':'As Saint-Étienne','strasbourg':'Rc Strasbourg Alsace','stuttgart':'Vfb Stuttgart',
    'torino':'Torino Fc','tottenham':'Tottenham Hotspur','toulouse':'Fc Toulouse','udinese':'Udinese Calcio',
    'union berlin':'1.Fc Union Berlin','valencia':'Valencia Cf','valladolid':'Real Valladolid Cf',
    'venezia':'Venezia Fc','villarreal':'Villarreal Cf','werder bremen':'Sv Werder Bremen',
    'west ham':'West Ham United','wolfsburg':'Vfl Wolfsburg','wolverhampton':'Wolverhampton Wanderers'
}

# === 1) LOAD & CLEAN COLUMN NAMES ===
df = pd.read_excel(INPUT_FILE, engine="openpyxl")
df.columns = (
    df.columns
      .str.lower()
      .str.strip()
      .str.replace(r'[^\w]+','_',regex=True)
      .str.replace(r'__+','_',regex=True)
)

# === 2) TEXT CLEANING & MAPPING ===
if 'player' in df: df['player'] = df['player'].apply(clean_text)
if 'nation' in df: df['nation'] = map_series(df['nation'], nation_mapping).str.title()
if 'squad'  in df: df['squad']  = map_series(df['squad'],  club_mapping).str.title()

# === 3) FIX NUMERIC & MISSING ===
# playing_time_min ← matches_played*90 if missing
if 'playing_time_min' in df and 'matches_played' in df:
    df['playing_time_min'] = df['playing_time_min'].fillna(df['matches_played']*90)

# salary numeric
if 'gross_p_w_(eur)' in df:
    df['gross_p_w_(eur)'] = pd.to_numeric(df['gross_p_w_(eur)'],errors='coerce').fillna(0)

# age_stats ← 2025 - born if missing
if 'born' in df and 'age_stats' in df:
    df['age_stats'] = df['age_stats'].fillna(2025 - pd.to_numeric(df['born'],errors='coerce'))

# season unify
if 'season_' in df and 'season' in df:
    df['season'] = df['season'].fillna(df['season_'])
elif 'season_' in df:
    df['season'] = df['season_']
df['season'] = df['season'].astype(str)

# === 4) PROXY MARKET VALUE & LAG ===
perf_col = 'performance_index' if 'performance_index' in df else 'rating_10'
if perf_col in df:
    df['proxy_market_value'] = (
        df['gross_p_w_(eur)']*52 +
        df[perf_col]*10000 +
        df['age_stats']*1000
    )
else:
    df['proxy_market_value'] = df['gross_p_w_(eur)']*52

df = df.sort_values(['player','season'])
df['market_value_lag'] = df.groupby('player')['proxy_market_value'].shift(1).fillna(0)

# === 5) TRANSFER STATUS ===
def transfer_status(gr):
    gr = gr.sort_values('season')
    prev_club = prev_season = None
    status = []
    for _,r in gr.iterrows():
        c, s = r['squad'], r['season']
        if prev_season==s and c!=prev_club:      status.append(1)
        elif prev_season and int(s.split('-')[0])==int(prev_season.split('-')[0])+1 and c!=prev_club:
            status.append(2)
        else: status.append(0)
        prev_club, prev_season = c,s
    return pd.Series(status,index=gr.index)

df['transfer_status'] = df.groupby('player',group_keys=False).apply(transfer_status)

# === 6) HOMEGROWN / LOW‐PAID FLAG ===
w_med = df['gross_p_w_(eur)'].median()
df['homegrown_flag'] = ((df['age_stats']<20)&(df['playing_time_min']>=1000)).astype(int)
df['low_paid_flag'] = (df['gross_p_w_(eur)']<w_med).astype(int)
df['homegrown_or_lowpaid'] = ((df['homegrown_flag']|df['low_paid_flag'])).astype(int)

# === 7) DEDUPE EXACT DUPLICATES ===
df = df.drop_duplicates()

# === 8) SAVE ===
df.to_excel(OUTPUT_FILE, index=False, engine="openpyxl")
print(f"✅ Done — cleaned data saved to {OUTPUT_FILE}")


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load your dataset
df = pd.read_excel("FINAL.xlsx")

# Filter out non-positive market values for log transformation
df = df[df['proxy_market_value'] > 0].copy()
df['log_market_value'] = np.log(df['proxy_market_value'])

# Drop rows with critical NAs
df = df.dropna(subset=[
    'log_market_value', 'performance_score', 'adj._gross_(eur)', 
    'age_stats', 'homegrown_low_paid_flag', 'transfer_status'
])

In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# === Load dataset ===
df = pd.read_excel("FINAL.xlsx")

# === Preprocessing ===
# Ensure only valid market values
df = df[df['proxy_market_value'] > 0].copy()

# Log transform the dependent variable
df['log_market_value'] = np.log(df['proxy_market_value'])

# Create interaction term: performance × wage
df['interaction_perf_wage'] = df['performance_score'] * df['adj._gross_(eur)']

# Drop rows with missing data for any key variable
model_df = df.dropna(subset=[
    'log_market_value',
    'performance_score',
    'adj._gross_(eur)',
    'age_stats',
    'homegrown_low_paid_flag',
    'transfer_status',
    'interaction_perf_wage'
])

# === Define predictors and response ===
X = model_df[[
    'performance_score',
    'adj._gross_(eur)',
    'homegrown_low_paid_flag',
    'transfer_status',
    'age_stats',
    'interaction_perf_wage'
]]
X = sm.add_constant(X)  # Adds intercept
y = model_df['log_market_value']

# === Run OLS Regression ===
model = sm.OLS(y, X).fit()

# === Output Summary ===
print(model.summary())

# === Save Output to File ===
with open("consolidated_regression_output.txt", "w") as f:
    f.write(model.summary().as_text())

print("✅ Regression complete. Output saved to 'consolidated_regression_output.txt'.")

                            OLS Regression Results                            
Dep. Variable:       log_market_value   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     4157.
Date:                Sat, 03 May 2025   Prob (F-statistic):               0.00
Time:                        01:15:33   Log-Likelihood:                -11579.
No. Observations:                8490   AIC:                         2.317e+04
Df Residuals:                    8483   BIC:                         2.322e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load dataset
df = pd.read_excel("FINAL.xlsx")

# Create interaction and lagged proxy value variables BEFORE dropna
df['performance_wage_interaction'] = df['performance_score'] * df['adj._gross_(eur)']
df['proxy_market_value_lag'] = df['adj._gross_(eur)'] * df['performance_score'] / (df['age_stats'] + 1)  # Example proxy

# Define variables used in the final regression
regression_vars = [
    'performance_score',
    'adj._gross_(eur)',
    'age_stats',
    'homegrown_low_paid_flag',
    'transfer_status',
    'performance_wage_interaction',
    'proxy_market_value_lag'
]

# Drop rows with missing values in those columns
df = df.dropna(subset=regression_vars + ['proxy_market_value'])

# Create log-transformed dependent variable
df['log_market_value'] = np.log(df['proxy_market_value'])

# Prepare feature matrix for VIF (excluding DV)
X = sm.add_constant(df[regression_vars])

# Compute VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

                       Variable        VIF
0                         const  51.980815
1             performance_score   1.696240
2              adj._gross_(eur)   2.914996
3                     age_stats   1.295236
4       homegrown_low_paid_flag   1.394459
5               transfer_status   1.010212
6  performance_wage_interaction  50.379915
7        proxy_market_value_lag  48.562542


/opt/anaconda3/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
import pandas as pd

# Load your cleaned DataFrame
file_path = "FINAL.xlsx"
df = pd.read_excel(file_path)

# Select relevant columns for modeling and anonymization
selected_cols = [
    'rating_10',
    'adj._gross_(eur)',
    'age_stats',
    'homegrown_low_paid_flag',
    'transfer_status',
    'proxy_market_value',
    'market_value_lag'
]

# Drop NA and sample 500 random rows
df_sample = df[selected_cols].dropna().sample(n=500, random_state=42)

# Export to CSV
df_sample.to_csv("anonymized_sample_2020_2025.csv", index=False)
print("✅ Sample anonymized dataset saved as 'anonymized_sample_2020_2025.csv'.")

✅ Sample anonymized dataset saved as 'anonymized_sample_2020_2025.csv'.
